In [1]:
import json
import logging
import os

# Set the logging level to desired verbosity
# This will help in debugging by showing detailed logs of the operations
# and interactions with the API.
from nexosapi.common.logging import setup_logging

LOGGING_LEVEL = logging.INFO
setup_logging(level=LOGGING_LEVEL)

# Set the environment variable to use dotenv to load environment variables
os.environ["NEXOSAI_INIT__LOAD_DOTENV"] = "true"

In [2]:
from nexosapi.api.endpoints import chat

# Start with a new request
tool_request = {
    "model": "7071fb76-722a-41ad-8477-68123b04b4ab",
    "messages": [
        {
            "role": "system",
            "content": """
            You are a helpful assistant that provides current currency prices
            based on the latest available data. You can use a Google search tool
            to find the most recent exchange rates. When asked for a currency price,
            you will search for the current price of the specified currency in the
            specified currency. For example, if asked for the price of an euro in Polish Zlot
            (PLN), you will search for "current price of euro in PLN" and provide the
            latest exchange rate you find. Provide the answer in JSON format with the
            following structure:
            {
                "currency": "EUR",
                "price_in_pln": 4.5
            }
            where "currency" is the currency you searched for and "price_in_pln" is the
            current price of that currency in Polish Zloty (PLN).
            If you cannot find the price, respond with an error message in JSON format:
            {
                "error": "Could not find the current price of the specified currency."
            }
        """,
        },
        {"role": "user", "content": "What is the current price of an euro in Polish Zloty (PLN)?"},
    ],
}
# Prepare the new request with the specified parameters
chat.completions.request.prepare(tool_request)

# Add a Google search tool to the request
chat.completions.request.with_search_engine_tool(
    options={
        "search_context_size": "medium",
    },
)

# Show the request before sending it
tool_request_to_send = chat.completions.request.dump()
logging.info(f"Request before sending: {json.dumps(tool_request_to_send, indent=2)}")

2025-08-07 00:35:25,077 - root - INFO - [SDK] Dependencies automatically wired.
2025-08-07 00:35:25,078 - root - INFO - Request before sending: {
  "model": "7071fb76-722a-41ad-8477-68123b04b4ab",
  "messages": [
    {
      "role": "system",
      "content": "\n            You are a helpful assistant that provides current currency prices\n            based on the latest available data. You can use a Google search tool\n            to find the most recent exchange rates. When asked for a currency price,\n            you will search for the current price of the specified currency in the\n            specified currency. For example, if asked for the price of an euro in Polish Zlot\n            (PLN), you will search for \"current price of euro in PLN\" and provide the\n            latest exchange rate you find. Provide the answer in JSON format with the\n            following structure:\n            {\n                \"currency\": \"EUR\",\n                \"price_in_pln\": 4.5\n       

In [3]:
# Send the new request to the chat completions endpoint with the Google search tool
chat.completions.request.reload_last()
tool_response = await chat.completions.request.send()

# Get the response from the chat completions endpoint
logging.info(f"Response: {tool_response.response_choices_as_text}")

2025-08-07 00:35:27,486 - httpx - INFO - HTTP Request: POST https://api.nexos.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-07 00:35:27,490 - root - INFO - Response: Choice 1:
  Message: {
    "currency": "EUR",
    "price_in_pln": 4.27
}
  Finish Reason: stop
  Index: 0
